In [38]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [39]:
#My Table:
class Player(Base):
    __tablename__ = 'player'
    id = Column(Integer, primary_key=True)
    player_name = Column(String)
    year = Column(Float)
    team = Column(String)
    ft = Column(Float)
    fg = Column(Float)
    assist = Column(Float)
    block = Column(Float)
    point = Column(Float)

# Table for you to load your summary. You can define the items you want.
class Census(Base):
    __tablename__ = 'census'
    id = Column(Integer, primary_key=True)


In [40]:
Base.metadata.tables

immutabledict({'player': Table('player', MetaData(bind=None), Column('id', Integer(), table=<player>, primary_key=True, nullable=False), Column('player_name', String(), table=<player>), Column('year', Float(), table=<player>), Column('team', String(), table=<player>), Column('ft', Float(), table=<player>), Column('fg', Float(), table=<player>), Column('assist', Float(), table=<player>), Column('block', Float(), table=<player>), Column('point', Float(), table=<player>), schema=None), 'census': Table('census', MetaData(bind=None), Column('id', Integer(), table=<census>, primary_key=True, nullable=False), schema=None)})

In [41]:
engine = create_engine('sqlite:///ETL_DB.sqlite')

In [42]:
Base.metadata.create_all(engine)

In [43]:
from sqlalchemy.orm import Session
session = Session(engine)

In [44]:
csv_file = "Seasons_Stats.csv"
player_data_df = pd.read_csv(csv_file)
kb_overall_data=player_data_df.loc[player_data_df["Player"]=="Kobe Bryant",:]

In [45]:
kb_specific=kb_overall_data[["Player","Year","Tm","FT%","FG%","AST","BLK","PTS"]]
kb_specific=kb_specific.reset_index(drop=True)

In [46]:
kb_specific=kb_specific.rename(columns={"Player":"player_name","Year":"year","Tm":"team",
                                        "FT%":"ft","FG%":"fg","AST":"assist","BLK":"block","PTS":"point"})
kb_specific.head()

,player_name,year,team,ft,fg,assist,block,point
0,Kobe Bryant,1997.0,LAL,0.819,0.417,91.0,23.0,539.0
1,Kobe Bryant,1998.0,LAL,0.794,0.428,199.0,40.0,1220.0
2,Kobe Bryant,1999.0,LAL,0.839,0.465,190.0,50.0,996.0
3,Kobe Bryant,2000.0,LAL,0.821,0.468,323.0,62.0,1485.0
4,Kobe Bryant,2001.0,LAL,0.853,0.464,338.0,43.0,1938.0


In [47]:
engine.table_names()

['census', 'player']

In [48]:
kb_specific.to_sql(name='player', con=engine, if_exists='append', index=False)

In [49]:
pd.read_sql_query('select * from player', con=engine).head()

,id,player_name,year,team,ft,fg,assist,block,point
0,1,Kobe Bryant,1997.0,LAL,0.819,0.417,91.0,23.0,539.0
1,2,Kobe Bryant,1998.0,LAL,0.794,0.428,199.0,40.0,1220.0
2,3,Kobe Bryant,1999.0,LAL,0.839,0.465,190.0,50.0,996.0
3,4,Kobe Bryant,2000.0,LAL,0.821,0.468,323.0,62.0,1485.0
4,5,Kobe Bryant,2001.0,LAL,0.853,0.464,338.0,43.0,1938.0
